# Dirty Dataset
This example illustrate how to perform meta-blocking on a dirty dataset (data deduplication), so when we have one dataset that contains duplicates.

In [1]:
import sparker
import random

## Load the data
sparkER provides wrappers to load CSV and JSON files.

*real_id_field* is the field that contains the identifier of the record.

In [2]:
# Profiles contained in the first dataset
profiles = sparker.CSVWrapper.load_profiles('../datasets/dirty/cora/cora.csv', real_id_field = "id")

Let's visualize a profile to check if they are correctly loaded

In [3]:
print(profiles.take(1)[0])

{'profile_id': 0, 'attributes': [{'key': 'venue', 'value': 'in proc. 36th annual symposium on foundations of computer science,'}, {'key': 'address', 'value': 'los alamitos, ca:'}, {'key': 'pages', 'value': 'pp. 322-331.'}, {'key': 'year', 'value': '1995,'}, {'key': 'author', 'value': 'p. auer, n. cesa-bianchi, y. freund, and r. e. schapire,'}, {'key': 'publisher', 'value': 'ieee computer society press,'}, {'key': 'title', 'value': "'gambling in a rigged casino: the adversarial multi-armed bandit problem,'"}], 'original_id': '0', 'source_id': 0}


Extract the max id (it will be used later)

In [4]:
# Max profile id
max_profile_id = profiles.map(lambda profile: profile.profile_id).max()

### Groundtruth (optional)
If you have a groundtruth you can measure the performance of each step.

When you load the groundtruth it contains the original profiles IDs, it is necessary to convert it to use the IDs assigned to each profile by Spark.

In [5]:
# Loads the groundtruth, takes as input the path of the file and the names of the attributes that represent
# respectively the id of profiles of the first dataset and the id of profiles of the second dataset
gt = sparker.CSVWrapper.load_groundtruth('../datasets/dirty/cora/groundtruth.csv', 'id1', 'id2')

In [6]:
# Converts the groundtruth by replacing original IDs with those given by Spark
new_gt = sparker.Converters.convert_groundtruth(gt, profiles)

In [7]:
# We can explore some pairs
random.sample(new_gt, 2)

[(1176, 1204), (1181, 1197)]

## Blocking
Now we can perform blocking.

By default sparkER performs token blocking, but it is possible to provide a different blocking function.

In the following example each token is splitted in ngrams of size 4 that are used for blocking.

In [8]:
blocks = sparker.Blocking.create_blocks(profiles, 
                                        blocking_method=sparker.BlockingKeysStrategies.ngrams_blocking,
                                        ngram_size=4)
print("Number of blocks",blocks.count())

Number of blocks 3388


Let's continue by using the standard token blocking

In [9]:
blocks = sparker.Blocking.create_blocks(profiles)
print("Number of blocks",blocks.count())

Number of blocks 891


## Block cleaning

sparkER implements two block cleaning strategies:

* Block purging: discard the largest blocks that involve too many comparisons, the parameter must be >= 1. A lower value mean a more aggressive purging.
* Block cleaning: removes for every profile the largest blocks in which it appears. The parameter is in range ]0, 1\[. A lower value mean a more aggressive cleaning.

In [10]:
# Perfoms the purging
blocks_purged = sparker.BlockPurging.block_purging(blocks, 1.025)

In [11]:
# Performs the cleaning
(profile_blocks, profile_blocks_filtered, blocks_after_filtering) = sparker.BlockFiltering.block_filtering_quick(blocks_purged, 0.8)

If you have the groundtruth, after every blocking step it is possible to check which are the performance of the blocking collection.

In [12]:
recall, precision, cmp_n = sparker.Utils.get_statistics(blocks_after_filtering, max_profile_id, new_gt)

print("Recall", recall)
print("Precision", precision)
print("Number of comparisons", cmp_n)

Recall 0.9966829608938548
Precision 0.08624431609319845
Number of comparisons 198587


## Meta-blocking
Meta-blocking can be used to further refine the block collection removing superfluous comparisons.

SparkER implements different kind of meta-blocking algorithms, you can find the descriptions in our paper.


For every partition of the RDD the pruning algorithm returns as output a triplet that contains:

* The number of edges
* The number of matches (only if the groundtruth is provided)
* The retained edges

To perform the meta-blocking first some data structures have to be created.

In [13]:
block_index_map = blocks_after_filtering.map(lambda b : (b.block_id, b.profiles)).collectAsMap()
block_index = sc.broadcast(block_index_map)

# This is only needed for certain weight measures
profile_blocks_size_index = sc.broadcast(profile_blocks_filtered.map(lambda pb : (pb.profile_id, len(pb.blocks))).collectAsMap())

# Broadcasted groundtruth
gt_broadcast = sc.broadcast(new_gt)

### Weighted Node Pruning

In [14]:
results = sparker.WNP.wnp(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.OR
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
print("Recall", num_matches/len(new_gt))
print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Recall 0.982483705772812
Precision 0.28767870226796394
Number of comparisons 58687


### Reciprocal Weighted Node Pruning

In [15]:
results = sparker.WNP.wnp(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.AND
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
print("Recall", num_matches/len(new_gt))
print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Recall 0.9785265363128491
Precision 0.40751781299985457
Number of comparisons 41262


### Weighted Edge Pruning

In [16]:
results = sparker.WEP.wep(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
print("Recall", num_matches/len(new_gt))
print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Recall 0.9784683426443203
Precision 0.447478376580173
Number of comparisons 37575


### Cardinality Node Pruning

In [17]:
results = sparker.CNP.cnp(
                          blocks_after_filtering,
                          profiles.count(),
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.OR
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
print("Recall", num_matches/len(new_gt))
print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Recall 0.4684590316573557
Precision 0.8009950248756219
Number of comparisons 10050


### Reciprocal Cardinality Node Pruning

In [18]:
results = sparker.CNP.cnp(
                          blocks_after_filtering,
                          profiles.count(),
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index,
                          comparison_type=sparker.ComparisonTypes.AND
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
print("Recall", num_matches/len(new_gt))
print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Recall 0.23062150837988826
Precision 0.9384323940326782
Number of comparisons 4223


### Cardinality Edge Pruning

In [19]:
results = sparker.CEP.cep(
                          profile_blocks_filtered,
                          block_index,
                          max_profile_id,
                          weight_type=sparker.WeightTypes.CBS,
                          groundtruth=gt_broadcast,
                          profile_blocks_size_index=profile_blocks_size_index
                         )
num_edges = results.map(lambda x: x[0]).sum()
num_matches = results.map(lambda x: x[1]).sum()
print("Recall", num_matches/len(new_gt))
print("Precision", num_matches/num_edges)
print("Number of comparisons",num_edges)

Recall 0.46019553072625696
Precision 0.9280600868442671
Number of comparisons 8521


## Collecting edges after meta-blocking
As mentioned before, the third element of the tuples returned by the meta-blocking contains the edges.


Edges are weighted according to the weight strategy provided to the meta-blocking.

In [20]:
edges = results.flatMap(lambda x: x[2])

edges.take(10)

[(0, 785, 8),
 (0, 787, 8),
 (0, 790, 8),
 (0, 794, 8),
 (0, 442, 8),
 (15, 17, 20),
 (15, 30, 17),
 (15, 31, 20),
 (15, 32, 13),
 (15, 35, 11)]